# MongoDB更新与删除

In [1]:
from pymongo import MongoClient
from datetime import datetime
from bson import ObjectId

In [2]:
client = MongoClient('localhost', 27017)

In [109]:
for i, db in enumerate(client.list_databases()):
    if i == 0:
        for key in db:
            print(f"{key:<25s}", end='')
        print()
    for key in db:
        if isinstance(db[key], float):
            print(f"{db[key]:<25.0f}", end='')
        elif isinstance(db[key], bool):
            print(f"{db[key]:<25}", end='')
        else:
            print(f"{str(db[key]):<25}", end='')
    print()
        

name                     sizeOnDisk               empty                    
abmDiffusion             11501568                 0                        
admin                    32768                    0                        
config                   110592                   0                        
lagou                    950272                   0                        
local                    77824                    0                        
mdb_test                 421888                   0                        
moniter_crowdfunding     147832832                0                        
purchase                 606208                   0                        
temp                     73728                    0                        
test                     40960                    0                        


## 一、更新文档

`db.collection.update_one()`或`db.collection.update_many()` 方法用于更新已存在的文档。语法格式如下：
```javascript
db.<collection_name>.update_one(
   <query>,
   <update>,
   {
     upsert: <boolean>,
     multi: <boolean>,
     writeConcern: <document>
   }
)
```
>参数说明：
- `query: update`的查询条件，类似`sql update`查询内`where`子句。
- `update: update`的对象和一些更新的操作符（如`$,$inc`...）等，也可以理解为`sql update`查询内`set`子句
- `upsert`: 可选, 即如果不存在`update`的记录，是否插入`objNew`,　**`True`为插入，默认是`False`，不插入**。
- `multi`: 可选，默认是`False`, 只更新找到的第1条记录, 如果这个参数为`True`,就把按条件查出来多条记录全部更新。
- `writeConcern` :可选，抛出异常的级别。

In [111]:
db = client.mdb_test

In [112]:
db.list_collection_names()

['bios', 'collec_test', 'tags', 'users', 'scores', 'products', 'student']

In [113]:
for x in db.student.find():
    print(x)

{'_id': ObjectId('5cd9273cf5d4321faa2849ec'), 'student_id': 2015210315, 'student_name': '张三丰', 'gender': '男'}
{'_id': ObjectId('5cd92796f5d4321faa2849ed'), 'student_id': 2015210316, 'student_name': '李想', 'gender': '女'}
{'_id': 2015210317, 'student_name': '郭涛', 'gender': '男'}
{'_id': ObjectId('5cd9294af5d4321faa2849ee'), 'student_id': 2015210318, 'student_name': 'Joshi', 'gender': '男'}
{'_id': ObjectId('5cd9294af5d4321faa2849ef'), 'student_id': 2015210319, 'student_name': 'Kevin', 'gender': '男'}
{'_id': ObjectId('5cd9294af5d4321faa2849f0'), 'student_id': 2015210320, 'student_name': 'Lily', 'gender': '女'}
{'_id': ObjectId('5cd9352af5d4321faa2849f1'), 'student_id': 2015210321, 'student_name': 'Steven', 'gender': '男', 'score': [70, 89]}
{'_id': ObjectId('5cd9352af5d4321faa2849f2'), 'student_id': 2015210322, 'student_name': 'Tony', 'gender': '男', 'score': [65, 89]}
{'_id': ObjectId('5cd9928af5d4321faa2849f3'), 'student_id': 2015210323, 'student_name': 'Tom', 'gender': '男', 'score': [65, 89,

### 域更新操作符

以下是一些常用的更新操作符.

| 名称     | 描述     |
| ---- | ---- |
| `$currentDate`       | 将目标`field`的值设置为当前日期; 如果目标`field`不存在, 将目标域加入`document`      |
| `$inc`  | 将目标`field`的值增加特定的量  |
| `$min`   | 当目标值小于`field`值时, 更新为目标值; 否则保留原有`field`值不变  |
| `$max`   | 当目标值大于`field`值时, 更新为目标值; 否则保留原有`field`值不变  |
| `$mul`   | 将`field`的值乘以目标值  |
| `$rename`   | 重命名`field`  |
| `$set`   | 为`field`设为目标值  |
| `$setOnInsert`   | 为更新时新插入文档的`field`设置目标值, 如果更新前文档已存在, 则不操作  |
| `$unset`   | 删除文档中的特定`field`; 如果目标`field`不存在, 则不进行任何操作  |


### 1. `$set`更新符

将符合条件的文档的某个域对应的值更新

In [40]:
db.student.update_one({"student_name": '李想'}, {'$set': {"gender": '女'}})  # 将student_name为李想的文档的gender更新为女

In [41]:
x = db.student.find_one({"student_name": '李想'})
x

{'_id': ObjectId('5cd92796f5d4321faa2849ed'),
 'student_id': 2015210316,
 'student_name': '李想',
 'gender': '女'}

- 如果更新的时候`<query>`未匹配任何记录, 则可通过指定`{"upsert":True}`将该记录插入进目标`collection`.

In [46]:
db.student.update_one({"student_name": '张山峰'}, {'$set': {"gender": '男'}}) 
db.student.find_one({"student_name": '张山峰'})

> 以上尝试将student_name为张山峰的文档的gender更新为男，然而由于student中没有名字为张山峰的学生，因此不更新任何文档（不会报错）。如果加入`{'upsert': True}`则可将`{"_id": ...., "student_name": '张山峰', "gender": '男'}`加入`student`中。

In [49]:
db.student.update_one({"student_name": '张山峰'},  {'$set': {"gender": '男'}}, upsert=True)

In [50]:
db.student.find_one({"student_name": '张山峰'})

{'_id': ObjectId('5ce24edc63dd06c18464d2cc'),
 'student_name': '张山峰',
 'gender': '男'}

- `$setOnInsert` 在`update=True`时，设置除`$set`之外的其它域值对。

```python
db.collection.update(
   <query>,
   {'$set': {<field2>: <value2>}
    '$setOnInsert': { <field2>: <value2>, ... } },
  upsert=True}
)
```

In [142]:
db.student.update_one({"student_name": '张勇'},  {'$set': {"gender": '男'}, '$setOnInsert': {"student_id": 2015210999}}, upsert=True)

#### 2. `$currentDate`更新符

In [57]:
from datetime import datetime

示例: 往集合`users`插入一个document: `{ _id: 1, status: "a", lastModified: ISODate("2013-10-02T01:11:18.965Z") }`, 然后利用`$currentDate`操作符更新`lastModified`的值

In [71]:
db.users.insert_one({'_id': 1, 'status': "a", 'lastModified': datetime(2013, 10, 2, 1, 11, 18, 965)})

In [72]:
db.users.find_one()

{'_id': 1,
 'status': 'a',
 'lastModified': datetime.datetime(2013, 10, 2, 1, 11, 18)}

In [73]:
db.users.update_one(
    {'_id': 1},
    {
        '$currentDate': {
            'lastModified': True,
            'cancellation.date': {'$type': 'timestamp'}
        },
        '$set': {
            'status': 'D',
            'cancellation.reason': "user request"}
    },
)

In [75]:
db.users.find_one()

{'_id': 1,
 'status': 'D',
 'lastModified': datetime.datetime(2019, 5, 20, 7, 36, 6, 760000),
 'cancellation': {'date': Timestamp(1558337766, 1), 'reason': 'user request'}}

In [69]:
db.users.delete_one({"_id": 1})

#### 3. `$inc`更新符
语法格式如下:`{ '$inc': { <field1>: <amount1>, <field2>: <amount2>, ... } }`. `$inc`操作符接收一个正数或者负数. 如果目标`field`不存在, 则插入该`field`和`$inc`目标值.

示例: 往集合`products`插入一个document: `{'_id': 1, 'sku': "abc123", 'quantity': 10, 'metrics': {'orders': 2, 'ratings': 3.5}}`, 然后利用`$inc`操作符更新`quantity`和`metrics.orders`的值.

In [81]:
db.products.insert_one({'_id': 1, 'sku': "abc123", 'quantity': 10, 'metrics': {'orders': 2, 'ratings': 3.5}})

In [84]:
db.products.find_one()

{'_id': 1,
 'sku': 'abc123',
 'quantity': 8,
 'metrics': {'orders': 3, 'ratings': 3.5}}

In [83]:
db.products.update_one(
    {"sku": "abc123"},
    {
        "$inc": {
            "quantity": -2, "metrics.orders": 1
        }
    }
)

In [80]:
db.products.delete_one({"_id":1})

#### 4. `$max`和`$min`更新符

语法格式: `{ $min: { <field1>: <value1>, ... } }` 或者 `{ $max: { <field1>: <value1>, ... } }`. 这两个操作符可用于数值和日期比较.

- 更新数值

往scores集合中插入一条记录`{ _id: 1, highScore: 800, lowScore: 200 }`, 然后利用`$max`和`$min`对其`lowerScore`和`higherScore`进行更新操作.

In [85]:
db.scores.insert_one({'_id': 1, 'highScore': 800, 'lowScore': 200 })

In [86]:
db.scores.update_one({'_id': 1}, 
                     {"$min": {'highScore': 900}, 
                      "$max": {'lowScore': 250}
                     })

In [87]:
db.scores.find_one()

{'_id': 1, 'highScore': 800, 'lowScore': 250}

- 更新时间

往tags集合中插入一个文档`{ _id: 1, desc: "crafts", dateEntered: ISODate("2013-10-01T05:00:00Z"), dateExpired: ISODate("2013-10-01T16:38:16Z") }`, 然后利用`$max`和`$min`对其进行更新操作.

In [95]:
db.tags.insert_one({
  '_id': 1,
  'desc': "crafts",
  'dateEntered': datetime(2013, 10, 1, 5),
  'dateExpired': datetime(2015, 10, 1, 16, 38, 16)
})

In [96]:
db.tags.update_one(
   {'_id': 1 },
   {"$min": {'dateEntered': datetime(2013, 9, 25)},
    '$max': {'dateExpired': datetime(2016, 10, 1)}})

In [97]:
db.tags.find_one()

{'_id': 1,
 'desc': 'crafts',
 'dateEntered': datetime.datetime(2013, 9, 25, 0, 0),
 'dateExpired': datetime.datetime(2016, 10, 1, 0, 0)}

In [94]:
db.tags.delete_one({})

#### 5. `$mul`更新符

语法格式: `{ '$mul': { <field1>: <number1>, ... } }`. `field`必须包含一个数值.

**示例: 往products集合中插入一个文档`{ "_id" : 2, "item" : "ABC", "price" : NumberDecimal("10.99"), "qty" : 25 }`, 然后利用`$mul`进行操作.**

In [98]:
db.products.insert_one({ 
    "_id" : 2, 
    "item" : "ABC", 
    "price" : 10.99, 
    "qty" : 25 })

In [99]:
db.products.update_one({'_id': 2},
                      {
                          '$mul': {'price': 1.5, 
                                   'qty': 2}
                      })

In [101]:
db.products.find_one({'_id': 2})

{'_id': 2, 'item': 'ABC', 'price': 16.485, 'qty': 50}

#### 6. `$unset`更新符
语法格式: `{'$unset': { <field1>: "", <field2>: "", ... } }`. 注意, field可以对应任意值. 如果目标field不存在, 则不进行任何操作.

**示例: 删除products集合中_id为1的文档中的metrics域.**

In [102]:
for x in db.products.find({}):
    print(x)

{'_id': 1, 'sku': 'abc123', 'quantity': 8, 'metrics': {'orders': 3, 'ratings': 3.5}}
{'_id': 2, 'item': 'ABC', 'price': 16.485, 'qty': 50}


In [103]:
db.products.update_one({'_id':1}, 
                       {'$unset':{'metrics':0}})

In [104]:
for x in db.products.find({}):
    print(x)

{'_id': 1, 'sku': 'abc123', 'quantity': 8}
{'_id': 2, 'item': 'ABC', 'price': 16.485, 'qty': 50}


#### 7. `$rename`更新符
语法格式: `{'$rename': { <field1>: <newName1>, <field2>: <newName2>, ... } }`. 注意, 新的名称必须与原名称不同. 在逻辑上, `$rename`等价于通过`$unset`操作符删除掉原有域, 然后再插入一个有`newName`和原值的文档. 如果目标`field`不存在, 则不进行任何操作.

**示例: 往products集合中插入一个文档`{"_id" : 4, "item" : "apple", "price" : 16, "qty" : 45}`, 然后将item重命名为`product_name`。**

In [105]:
db.products.insert_one({"_id" : 4, "item" : "apple", "price" : 16, "qty" : 45})

In [106]:
for x in db.products.find({}):
    print(x)

{'_id': 1, 'sku': 'abc123', 'quantity': 8}
{'_id': 2, 'item': 'ABC', 'price': 16.485, 'qty': 50}
{'_id': 4, 'item': 'apple', 'price': 16, 'qty': 45}


In [107]:
db.products.update_one({'_id': 4}, {'$rename':{'item': "product_name"}})

In [108]:
db.products.find_one({'_id': 4})

{'_id': 4, 'price': 16, 'qty': 45, 'product_name': 'apple'}

### 其它更新方法

- `update_many(filter, update, upsert=False, array_filters=None, bypass_document_validation=False, collation=None, session=None)`。除了`update_one()`之外，也可以使用`update_many()`更新满足`filter`条件的1到多个文档。
- `find_one_and_update(filter, update, projection=None, sort=None, upsert=False, return_document=False, array_filters=None, session=None, **kwargs) `。`find_one_and_update`可用于查询与更新满足`filter`条件的文档。

## 二、删除文档

MongoDB中的集合有`delete_one(), delte_many(), find_one_and_delete()`等方法删除符合查询条件的文档。

### 1. `delte_one()`删除符合`filter`条件的文档

语法:
```python
db.collection.delete_one(
   <filter>,
   {
      writeConcern: <document>,
      collation: <document>
   }
```
- `<filter>`是必选参数，为文档类型，通过查询操作符指定操作类型，然后删除满足条件的第1个文档。

**示例: 删除products表中第1条文档，删除products中item为ABC的文档。**

In [114]:
list(db.products.find())

[{'_id': 1, 'sku': 'abc123', 'quantity': 8},
 {'_id': 2, 'item': 'ABC', 'price': 16.485, 'qty': 50},
 {'_id': 4, 'price': 16, 'qty': 45, 'product_name': 'apple'}]

In [117]:
db.products.delete_one({})

In [120]:
list(db.products.find())

[{'_id': 2, 'item': 'ABC', 'price': 16.485, 'qty': 50},
 {'_id': 4, 'price': 16, 'qty': 45, 'product_name': 'apple'},
 {'_id': 3, 'item': 'ABC', 'price': 16.485, 'qty': 50}]

In [119]:
db.products.insert_one({'_id': 3, 'item': 'ABC', 'price': 16.485, 'qty': 50})  # 先插入1条item为ABC的文档

In [121]:
db.products.delete_one({'item': 'ABC'})

In [122]:
list(db.products.find())  # 删除了1条item为ABC的文档

[{'_id': 4, 'price': 16, 'qty': 45, 'product_name': 'apple'},
 {'_id': 3, 'item': 'ABC', 'price': 16.485, 'qty': 50}]

### 2. `delete_many()`删除多个文档

语法:

```python
db.collection.delete_many(
   <filter>,
   {
      writeConcern: <document>,
      collation: <document>
   }
)
```

- `<filter>`为文档类型，通过查询操作符指定操作类型，然后删除满足条件的所有文档。该参数必须有值，即使是{}。如果要删除所有文档，则需传递一个空文档{}。

**示例: 删除products表所有item为"ABC"的记录。**

In [123]:
db.products.insert_one({'_id': 2, 'item': 'ABC', 'price': 16.485, 'qty': 50})  # 重新插入一条文档

In [124]:
list(db.products.find())

[{'_id': 4, 'price': 16, 'qty': 45, 'product_name': 'apple'},
 {'_id': 3, 'item': 'ABC', 'price': 16.485, 'qty': 50},
 {'_id': 2, 'item': 'ABC', 'price': 16.485, 'qty': 50}]

In [125]:
db.products.delete_many({'item': 'ABC'})

In [126]:
list(db.products.find())

[{'_id': 4, 'price': 16, 'qty': 45, 'product_name': 'apple'}]

### 3. `find_one_and_modify()`找到满足查询条件的第一条记录，然后将其删除，返回被删除的文档。

语法:

```python
db.collection.find_one_and_delete(
   <filter>,
   projection = {field: value, ...},
   sort = [(field1, 1), ...]
)
```

- `<filter>`是必选参数，文档类型，通过查询操作符指定操作类型，然后删除满足条件的第1条文档。该参数必须有值，即使是{}。
- `projection`为可选参数，文档类型，指定返回的文档包含哪些域。
- `sort`为可选参数，文档类型，指定了由filter指定的集合的排序。

**示例: 删除student集合中gender为男的第1条记录，其中，filter按照student_name降序排序，返回的文档包含student_id, student_name和gender。**

In [127]:
for x in db.student.find():
    print(x)

{'_id': ObjectId('5cd9273cf5d4321faa2849ec'), 'student_id': 2015210315, 'student_name': '张三丰', 'gender': '男'}
{'_id': ObjectId('5cd92796f5d4321faa2849ed'), 'student_id': 2015210316, 'student_name': '李想', 'gender': '女'}
{'_id': 2015210317, 'student_name': '郭涛', 'gender': '男'}
{'_id': ObjectId('5cd9294af5d4321faa2849ee'), 'student_id': 2015210318, 'student_name': 'Joshi', 'gender': '男'}
{'_id': ObjectId('5cd9294af5d4321faa2849ef'), 'student_id': 2015210319, 'student_name': 'Kevin', 'gender': '男'}
{'_id': ObjectId('5cd9294af5d4321faa2849f0'), 'student_id': 2015210320, 'student_name': 'Lily', 'gender': '女'}
{'_id': ObjectId('5cd9352af5d4321faa2849f1'), 'student_id': 2015210321, 'student_name': 'Steven', 'gender': '男', 'score': [70, 89]}
{'_id': ObjectId('5cd9352af5d4321faa2849f2'), 'student_id': 2015210322, 'student_name': 'Tony', 'gender': '男', 'score': [65, 89]}
{'_id': ObjectId('5cd9928af5d4321faa2849f3'), 'student_id': 2015210323, 'student_name': 'Tom', 'gender': '男', 'score': [65, 89,

In [133]:
db.student.find_one_and_delete({'gender':"男"}, sort=[('student_name', 1)], projection={'_id':False})

{'student_id': 2015210319, 'student_name': 'Kevin', 'gender': '男'}